In [24]:
import sys, os, django
import pandas as pd

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

from pyparsing import *

q = Query.objects.get(pk=7638)
t = Tag.objects.get(pk=4445)
t = Tag.objects.get(pk=4366)

#q = Query.objects.get(pk=7507)
#t = Tag.objects.get(pk=4329)


In [27]:
doids = set(DocOwnership.objects.filter(tag=t).values_list('doc__id',flat=True))
uids = set(DocOwnership.objects.filter(tag=t).values_list('user__id',flat=True))

users = User.objects.filter(pk__in=uids)

docs = Doc.objects.filter(pk__in=doids)#, docproject__project=q.project, docproject__relevant__in=[1,3])
db2 = Category.objects.get(id=362)
print(db2)

docs.count()

Does this document fulfill all Level 1 AND 2 criteria for inclusion in Database 2<hidden><wholetext>


100

In [25]:
tagid=t.id

In [11]:
#%%timeit -r 1 -n 1

t = Tag.objects.get(pk=tagid)
q = t.query
doids = set(DocOwnership.objects.filter(tag=t,relevant__gt=0).values_list('doc__id',flat=True))
uids = set(DocOwnership.objects.filter(tag=t).values_list('user__id',flat=True))

users = User.objects.filter(pk__in=uids)

docs = Doc.objects.filter(pk__in=doids)#, docproject__project=q.project, docproject__relevant__in=[1,3])
db2 = Category.objects.get(id=362)
doc_dict = []

for d in docs.values("title","content","id"):
    db1_cols = []
    db2_cols = []
    note_cols = []
    d = dict(d)
    db2s = []
    d["link"] = f"https://apsis.mcc-berlin.net/scoping/document/193/{d['id']}/"
    d["db1_overall"] = DocProject.objects.get(project=q.project, doc_id=d["id"]).get_relevant_display()
    for u in users:
        uname = u.username.split("@")[0]
        db2_cols.append(f"{uname} db2")
        db1_cols.append(f"{uname} db1")
        note_cols.append(f"{uname} notes")

        try:
            d[f"{uname} db2"] = DocUserCat.objects.get(doc_id=d["id"], user=u, category__parent_category=362).category.name
        except:
            d[f"{uname} db2"] = "Unrated"
        db2s.append(d[f"{uname} db2"])
        try:
            d[f"{uname} db1"] = DocOwnership.objects.get(doc_id=d["id"], user=u).get_relevant_display()
        except:
            d[f"{uname} db1"] = None
        try:
            d[f"{uname} notes"] = Note.objects.get(doc_id=d["id"], user=u).text
        except:
            d[f"{uname} notes"] = None

    if len(set(db2s))==1:
        db2v = db2s[0]
    else:
        db2v = "Mixed/Maybe"
    d["db2_overall"] = db2v
    doc_dict.append(d)
    
doc_df = pd.DataFrame.from_dict(doc_dict)


In [230]:
doc_info_df = pd.DataFrame.from_dict(docs.values('id','title','content'))
doc_info_df.head()

,content,id,title
0,Community-based climate change projects in the...,44683,Raising awareness about climate change in Paci...
1,This article examines the translation of key t...,49638,"Translating Climate Change: Adaptation, Resili..."
2,Decision making in the field of air quality an...,64887,A decision framework for Integrated Assessment...
3,In response to the increasing frequency and ec...,77756,Quantifying vulnerability of rural communities...
4,Land-use and climate change are expected to ca...,88908,The effect of long-term drying associated with...


In [4]:
def db_doc_table(docs,f,u,v,n,vmap=None,agg=True, fillna=None):
    df = pd.DataFrame.from_dict(
        docs.filter(**f).values('id', u, v)
    )

    df[u] = df[u].str.extract("(^[a-zA-Z0-9_.+-]*)") + f" - {n}"
    cols = df[u].unique()
    if vmap:
        df[v] = df[v].map(vmap)
        
    df = pd.DataFrame(df.groupby(["id",u]).apply(lambda x: "%s" % ', '.join(x[v]))).reset_index()
    df = df.rename(columns={0: v})
    df = df.pivot(index="id",columns=u,values=v).reset_index(drop=False)
    if agg:
        df[f'{n}_overall'] = df[cols].ffill(axis=1).iloc[:, -1]
        df.loc[df[cols].nunique(axis=1)>1,'db1_overall'] = "Mixed/Maybe"
    if fillna:
        df = df.fillna(fillna)
    return df

df = db_doc_table(
    docs,
    {"docownership__tag":t},
    "docownership__user__username",
    "docownership__relevant",
    "db1",
    vmap=dict(DocOwnership.Status),
    fillna="Unrated"
)

print(df.shape)
df.head()

(100, 6)


docownership__user__username,id,apfisch - db1,ee18ivc - db1,l.berrangford - db1,miriam - db1,db1_overall
0,44683,Maybe,Yes,Yes,Yes,Mixed/Maybe
1,49638,No,No,No,No,No
2,64887,No,No,No,No,No
3,77756,No,No,Yes,No,Mixed/Maybe
4,88908,No,No,No,No,No


In [5]:
f = {"note__project":t.query.project}
print(f)
u = "note__user__username"
v = "note__text"
n = "notes"
agg=False
df = pd.DataFrame.from_dict(
    docs.filter(**f).values('id', u, v)
)

df[u] = df[u].str.extract("(^[a-zA-Z0-9_.+-]*)") + f" - {n}"
cols = df[u].unique()
df = pd.DataFrame(df.groupby(["id",u]).apply(lambda x: "%s" % ', '.join(x[v]))).reset_index()
df = df.rename(columns={0: v})
df = df.pivot(index="id",columns=u,values=v).reset_index(drop=False)
df.head()

{'note__project': <Project: Global adaptation to climate change>}


note__user__username,id,apfisch - notes,l.berrangford - notes,miriam - notes
0,96092,NaN,NaN,can’t tell if this is empirical\r\nunder some ...
1,188512,NaN,NaN,"assessment, seems like just arguing for an ada..."
2,222427,NaN,NaN,non-human adaptation
3,264144,NaN,NaN,mitigation
4,412969,NaN,NaN,no abstract


In [39]:
def db_doc_table(docs,f,u,v,n,vmap=None,agg=True, fillna=None):
    df = pd.DataFrame.from_dict(
        docs.filter(**f).values('id', u, v)
    )
    if df.shape==(0,0):
        df = pd.DataFrame.from_dict(
            docs.values('id')
        )
        return df, []

    df[u] = df[u].str.extract("(^[a-zA-Z0-9_.+-]*)") + f" - {n}"
    cols = df[u].unique()
    if vmap:
        df[v] = df[v].map(vmap)
    df = pd.DataFrame(df.groupby(["id",u]).apply(lambda x: "%s" % ', '.join(x[v]))).reset_index()
    df = df.rename(columns={0: v})
    df = df.pivot(index="id",columns=u,values=v).reset_index(drop=False)
    if agg:
        df[f'{n}_overall'] = df[cols].ffill(axis=1).iloc[:, -1]
        df.loc[df[cols].nunique(axis=1)>1,f'{n}_overall'] = "Mixed/Maybe"
    if fillna:
        df = df.fillna(fillna)
    return df, cols

ex_df, ex_cols = db_doc_table(
    docs,
    {"docusercat__category__level__in": [7,8]},
    "docusercat__user__username",
    "docusercat__category__name",
    "exclusions",
    fillna="Unrated"
)

print(ex_df.shape)
ex_df.head(20)

(100, 1)


,id
0,44683
1,49638
2,64887
3,77756
4,88908
5,96092
6,110619
7,188512
8,222427
9,225751


In [41]:
l = list([])
l

[]

In [43]:
def db_doc_table(docs,f,u,v,n,vmap=None,agg=True, fillna=None):
    df = pd.DataFrame.from_dict(
        docs.filter(**f).values('id', u, v)
    )
    if df.shape==(0,0):
        df = pd.DataFrame.from_dict(
            docs.values('id')
        )
        return df, []
    df[u] = df[u].str.extract("(^[a-zA-Z0-9_.+-]*)") + f" - {n}"
    cols = df[u].unique()
    if vmap:
        df[v] = df[v].map(vmap)
    df = pd.DataFrame(df.groupby(["id",u]).apply(lambda x: "%s" % ', '.join(x[v]))).reset_index()
    df = df.rename(columns={0: v})
    df = df.pivot(index="id",columns=u,values=v).reset_index(drop=False)
    if agg:
        df[f'{n}_overall'] = df[cols].ffill(axis=1).iloc[:, -1]
        df.loc[df[cols].nunique(axis=1)>1,f'{n}_overall'] = "Mixed/Maybe"
    if fillna:
        df = df.fillna(fillna)
    return df, cols

doc_info_df = pd.DataFrame.from_dict(docs.values('id','title','content'))

print(doc_info_df.shape)

db1_doc_df, db1_cols = db_doc_table(
    docs,
    {"docownership__tag":t},
    "docownership__user__username",
    "docownership__relevant",
    "db1",
    vmap=dict(DocOwnership.Status),
    fillna="Unrated"
)

db2_doc_df, db2_cols = db_doc_table(
    docs,
    {"docusercat__category__parent_category":362},
    "docusercat__user__username",
    "docusercat__category__name",
    "db2",
    fillna="Unrated"
)

note_doc_df, note_cols = db_doc_table(
    docs,
    {"note__project":t.query.project},
    "note__user__username",
    "note__text",
    "notes",
    agg=False
)



(100, 3)


note__user__username,id,apfisch - notes,l.berrangford - notes,miriam - notes
0,96092,NaN,NaN,can’t tell if this is empirical\r\nunder some ...
1,188512,NaN,NaN,"assessment, seems like just arguing for an ada..."
2,222427,NaN,NaN,non-human adaptation
3,264144,NaN,NaN,mitigation
4,412969,NaN,NaN,no abstract
5,419130,"not empirical, and focusing on factor in ...","Unclear if empirical. Refers to analysis, but ...",not sure if counts as empirical
6,441306,"not empirical, proposing empirical?",Not clear if this is empirical. Book chapter.,Book Chapter.\r\nAbstract does not make clear ...
7,470715,doesn't seem to report on observed/documented ...,"Uses term 'mitigation', but actually seems mor...",maybe too mitigation and impacts focused?
8,484080,"not CC, and not observed responses ...",NaN,NaN
9,649460,NaN,NaN,Is this just AR5?


In [279]:
doc_df['link'] = "https://apsis.mcc-berlin.net/scoping/document/193/" + doc_df['id'].astype(str) 
doc_df.head()

,content,id,title,apfisch - db1,ee18ivc - db1,l.berrangford - db1,miriam - db1,db1_overall,ee18ivc - db2,l.berrangford - db2,miriam - db2,db2_overall,ee18ivc - notes,l.berrangford - notes,miriam - notes,link
0,Watersheds and rivers are vital ecological fea...,1373,Understanding watershed dynamics and impacts o...,Unrated,Yes,Unrated,Unrated,Yes,Maybe,Unrated,Unrated,Maybe,,,,https://apsis.mcc-berlin.net/scoping/document/...
1,"Many coasts are eroding. In the Comoros, as in...",3095,Seawalls as a response to coastal erosion and ...,Unrated,Unrated,Yes,Unrated,Yes,Unrated,No,Unrated,No,,DB2: no tangible response,,https://apsis.mcc-berlin.net/scoping/document/...
2,"Since the 1970s, the Niger basin has been char...",3309,Hydro-climatic changes in the Niger basin and ...,Unrated,Unrated,Unrated,Yes,Yes,Unrated,Unrated,No,No,,,about perceptions\r\nhistorical data analyzed ...,https://apsis.mcc-berlin.net/scoping/document/...
3,The heterogeneous topography of the Great Basi...,8968,Life in an extreme environment: a historical p...,No,Unrated,Unrated,Unrated,No,Unrated,Unrated,Unrated,Unrated,,,,https://apsis.mcc-berlin.net/scoping/document/...
4,Much of the population in the developing world...,13584,Environmental risk factors and child nutrition...,Unrated,No,Unrated,Unrated,No,Unrated,Unrated,Unrated,Unrated,,,,https://apsis.mcc-berlin.net/scoping/document/...


In [40]:
doc_df = doc_info_df.merge(
    db1_doc_df, how="left"
).merge(
    db2_doc_df, how="left"
).merge(
    note_doc_df, how="left"
).merge(
    ex_df, how="left"
)

fillcs = [x for x in doc_df.columns if " - notes" not in x]

doc_df[fillcs] = doc_df[fillcs].fillna("Unrated")
doc_df = doc_df.fillna("")

print(doc_df.shape)

doc_df.head()

(100, 16)


,content,id,title,apfisch - db1,ee18ivc - db1,l.berrangford - db1,miriam - db1,db1_overall,apfisch - db2,ee18ivc - db2,l.berrangford - db2,miriam - db2,db2_overall,apfisch - notes,l.berrangford - notes,miriam - notes
0,Community-based climate change projects in the...,44683,Raising awareness about climate change in Paci...,Maybe,Yes,Yes,Yes,Mixed/Maybe,No,Yes,No,No,Mixed/Maybe,,,
1,This article examines the translation of key t...,49638,"Translating Climate Change: Adaptation, Resili...",No,No,No,No,No,Unrated,Unrated,Unrated,Unrated,Unrated,,,
2,Decision making in the field of air quality an...,64887,A decision framework for Integrated Assessment...,No,No,No,No,No,Unrated,Unrated,Unrated,Unrated,Unrated,,,
3,In response to the increasing frequency and ec...,77756,Quantifying vulnerability of rural communities...,No,No,Yes,No,Mixed/Maybe,Unrated,Unrated,No,Unrated,No,,,
4,Land-use and climate change are expected to ca...,88908,The effect of long-term drying associated with...,No,No,No,No,No,Unrated,Unrated,Unrated,Unrated,Unrated,,,


In [118]:
values = ['title','content']

u = "note__user__username"

note_doc_df = pd.DataFrame.from_dict(
    docs.filter(
        note__project=t.query.project
    ).values(
        'id',
        u,
        'note__text'
    )
)

note_doc_df[u] = note_doc_df[u].str.extract("(^[a-zA-Z0-9_.+-]*)") + " - notes"

note_doc_df = note_doc_df.pivot(
    index="id",columns="note__user__username",
    values="note__text"
).reset_index(drop=False)

print(note_doc_df.shape)
note_doc_df.head()

(231, 4)


note__user__username,id,ee18ivc - notes,l.berrangford - notes,miriam - notes
0,3095,NaN,DB2: no tangible response,NaN
1,3309,NaN,NaN,about perceptions\r\nhistorical data analyzed ...
2,13937,NaN,DB: not empirical; natural systems only,NaN
3,14062,NaN,NaN,not human-assisted adaptation
4,17423,NaN,DB1: not response oriented,NaN


In [81]:
doc_df = pd.merge(doc_info_df,note_doc_df, how="left")

print(doc_df.shape)
doc_df.head()

(307, 6)


,content,id,title,ee18ivc@leeds.ac.uk,l.berrangford@leeds.ac.uk,miriam@iri.columbia.edu
0,Watersheds and rivers are vital ecological fea...,1373,Understanding watershed dynamics and impacts o...,NaN,NaN,NaN
1,"Many coasts are eroding. In the Comoros, as in...",3095,Seawalls as a response to coastal erosion and ...,NaN,DB2: no tangible response,NaN
2,"Since the 1970s, the Niger basin has been char...",3309,Hydro-climatic changes in the Niger basin and ...,NaN,NaN,about perceptions\r\nhistorical data analyzed ...
3,The heterogeneous topography of the Great Basi...,8968,Life in an extreme environment: a historical p...,NaN,NaN,NaN
4,Much of the population in the developing world...,13584,Environmental risk factors and child nutrition...,NaN,NaN,NaN


In [25]:
doc_df = pd.DataFrame.from_dict(doc_dict)
print(doc_df.shape)
doc_df.head()

(307, 18)


,apfisch db1,apfisch db2,apfisch notes,content,db1_overall,db2_overall,ee18ivc db1,ee18ivc db2,ee18ivc notes,id,l.berrangford db1,l.berrangford db2,l.berrangford notes,link,miriam db1,miriam db2,miriam notes,title
0,None,Unrated,None,Watersheds and rivers are vital ecological fea...,Yes,Mixed/Maybe,Yes,Maybe,None,1373,None,Unrated,None,https://apsis.mcc-berlin.net/scoping/document/...,None,Unrated,None,Understanding watershed dynamics and impacts o...
1,None,Unrated,None,"Many coasts are eroding. In the Comoros, as in...",Yes,Mixed/Maybe,None,Unrated,None,3095,Yes,No,DB2: no tangible response,https://apsis.mcc-berlin.net/scoping/document/...,None,Unrated,None,Seawalls as a response to coastal erosion and ...
2,None,Unrated,None,"Since the 1970s, the Niger basin has been char...",Yes,Mixed/Maybe,None,Unrated,None,3309,None,Unrated,None,https://apsis.mcc-berlin.net/scoping/document/...,Yes,No,about perceptions\r\nhistorical data analyzed ...,Hydro-climatic changes in the Niger basin and ...
3,No,Unrated,None,The heterogeneous topography of the Great Basi...,No,Unrated,None,Unrated,None,8968,None,Unrated,None,https://apsis.mcc-berlin.net/scoping/document/...,None,Unrated,None,Life in an extreme environment: a historical p...
4,None,Unrated,None,Much of the population in the developing world...,No,Unrated,No,Unrated,None,13584,None,Unrated,None,https://apsis.mcc-berlin.net/scoping/document/...,None,Unrated,None,Environmental risk factors and child nutrition...


In [155]:
writer = pd.ExcelWriter("screening-multiuser.xlsx", engine='xlsxwriter')

doc_df.to_excel(writer,index=False, startrow=1, header=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

cbase = ord('A')
tcols = ["id"]
wcols = ["title","content"]

wformat = workbook.add_format({
    'text_wrap': True,
    'valign': "top"
})

yformat = workbook.add_format({
    "fg_color": "#ffffb3"
})

gformat = workbook.add_format({
    "fg_color": "#8dd3c7"
})

bformat = workbook.add_format({
    "fg_color": "#80b1d3"
})

header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': 'black',
    'font_color': 'white',
    'border': 1})

for col_num, value in enumerate(doc_df.columns.values):
    worksheet.write(0, col_num , value, header_format)
    
worksheet.set_row(0,25)

for i, c in doc_df.iterrows():
    worksheet.set_row(i,75)

for i, c in enumerate(doc_df.columns):
    l = chr(cbase+i)
    if c == "title":
        worksheet.set_column(f"{l}:{l}",25, wformat)
    elif c == "content":
        worksheet.set_column(f"{l}:{l}",60, wformat)
    elif c in db1_cols:
        worksheet.set_column(f"{l}:{l}", 10, yformat )
    elif c in db2_cols:
        worksheet.set_column(f"{l}:{l}", 10, bformat )
    elif "_overall" in c:
        worksheet.set_column(f"{l}:{l}", 10, gformat )
    elif "notes" in c:
        worksheet.set_column(f"{l}:{l}",20, wformat)

writer.save()

In [106]:
doc_df.to_csv("screening_multiuser.csv",index=False)

In [170]:

docs = Doc.objects.filter(query=q,content__iregex="\w")
doids = DocOwnership.objects.filter(query=q).values_list('doc__id',flat=True)
docs = docs.exclude(pk__in=doids)
docs.count()

tag, created = Tag.objects.get_or_create(
    query=q,
    title="unassigned_with_abstract",
    text="Manually assigned"
)

docs = docs.exclude(tag=tag)

Through = Doc.tag.through
tms = [Through(doc_id=d,tag=tag) for d in set(docs.values_list('pk',flat=True))]
Through.objects.bulk_create(tms)

tag.update_tag()

In [168]:

docs = Doc.objects.filter(query=q,content__iregex="\w")
doids = DocOwnership.objects.filter(query=q,relevant__gt=0).values_list('doc__id',flat=True)
docs = docs.exclude(pk__in=doids)
docs.count()

tag, created = Tag.objects.get_or_create(
    query=q,
    title="unrated_with_abstract",
    text="Manually assigned"
)

docs = docs.exclude(tag=tag)

Through = Doc.tag.through
tms = [Through(doc_id=d,tag=tag) for d in set(docs.values_list('pk',flat=True))]
Through.objects.bulk_create(tms)

tag.update_tag()

56933